In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories and parameters
dataset_combined_dir = 'Mechanical Tools Image dataset/Mechanical Tools Image dataset'
target_size = (224, 224)  # Target image size for MobileNetV2
batch_size = 32
epochs = 10

In [2]:
# Use ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [3]:
# Load data from combined directory
train_generator = datagen.flow_from_directory(
    dataset_combined_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 7527 images belonging to 8 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(target_size[0], target_size[1], 3), include_top=False, weights='imagenet')

In [5]:
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [6]:
# Combine base model and custom top layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze layers in base model (optional)
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
model.fit(
    train_generator,
    epochs=10
)

Epoch 1/10


C:\Users\91819\Downloads\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


236/236 ━━━━━━━━━━━━━━━━━━━━ 259s 1s/step - accuracy: 0.7268 - loss: 0.8318
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 166s 692ms/step - accuracy: 0.8641 - loss: 0.3898
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 165s 687ms/step - accuracy: 0.8855 - loss: 0.3250
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 171s 707ms/step - accuracy: 0.8981 - loss: 0.2930
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 163s 678ms/step - accuracy: 0.9095 - loss: 0.2619
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 177s 737ms/step - accuracy: 0.9248 - loss: 0.2036
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 390s 2s/step - accuracy: 0.9335 - loss: 0.1843
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 518s 2s/step - accuracy: 0.9374 - loss: 0.1751
Epoch 9/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 547s 2s/step - accuracy: 0.9519 - loss: 0.1444
Epoch 10/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 9409s 40s/step - accuracy: 0.9452 - loss: 0.1378


In [12]:
# Save the trained model
model.save('tools_classification.keras')

In [11]:
#make predictions
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('tools_classification.keras')

# Define the labels for medicinal and toxic plants
labels = [
    "Gasoline Can", "Hammer", "Pebbles", "Pliers", "Rope","Screw Driver","Toolbox","Wrench"
]

# Function to preprocess an image
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)  # Load and resize image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch-like effect
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Function to predict and print category
def predict_and_print_category(image_path):
    img_array = preprocess_image(image_path)
    
    # Make predictions
    predictions = model.predict(img_array)
    
    # Print the predicted class probabilities
    print("Predicted class probabilities:", predictions)
    
    # Print the predicted class index
    predicted_class = np.argmax(predictions, axis=1)[0]
    print(f'Predicted class index: {predicted_class}')
   
    print(f"The image is classified as : {labels[predicted_class]}")
    
# Example: Predict on a new image from the combined dataset
new_image_path = 'Mechanical Tools Image dataset/Mechanical Tools Image dataset/Pebbels/000000.jpg'
predict_and_print_category(new_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class probabilities: [[2.5840029e-08 1.2696265e-04 9.9986947e-01 1.6923420e-06 1.0569247e-06
  2.4752760e-08 6.7595403e-07 3.3642827e-10]]
Predicted class index: 2
The image is classified as : Pebbles
